# **Project: Fine Tune LLM (GPT2) for medical dataset**


In this assignment I am using GPT-2 (124 M parameters). We can try using larger models as well (GPT2-medium, GPT2-large) but had to go forward with a small model due to resource constraints.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 106.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 124.4 MB/s eta 0:00:00


In [3]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 15.4 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import re
import os

# Data Preprocessing

I have collected the data from BioASQ collection of medical data. The data is availabe in json format and I formatted it in the following way to be able to use it while fine tuning our LLM. The train data has been made in the following way:

*   [Q] question1
*   [A] answer1
*   [Q] question2
*   [A] answer2

In [5]:
with open('/content/drive/MyDrive/Prezent.ai/train_data.txt', "r") as f:
    text_data = f.read()

In [6]:
# This is how the training data looks
print(text_data[0:600])

[Q] Is Hirschsprung disease a mendelian or a multifactorial disorder?
[A] Coding sequence mutations in RET, GDNF, EDNRB, EDN3, and SOX10 are involved in the development of Hirschsprung disease. The majority of these genes was shown to be related to Mendelian syndromic forms of Hirschsprung's disease, whereas the non-Mendelian inheritance of sporadic non-syndromic Hirschsprung disease proved to be complex; involvement of multiple loci was demonstrated in a multiplicative model.
[Q] List signaling molecules (ligands) that interact with the receptor EGFR?
[A] The 7 known EGFR ligands  are: epider


In [7]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [8]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

In [9]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator

In [10]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)
      
  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )
      
  trainer.train()
  trainer.save_model()


In [11]:
train_file_path = "/content/drive/MyDrive/Prezent.ai/train_data.txt"
model_name = 'gpt2' # here we can use a bigger model like gpt2-medium, gpt2-large etc
output_dir = '/content/drive/MyDrive/Prezent.ai/GPT-2/gpt-2-custom_q_and_a'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 24.0
save_steps = 50000

In [12]:
# Train
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.059700
1000,2.748700
1500,2.516000
2000,2.363400
2500,2.196800
3000,2.073600
3500,1.948500
4000,1.852600
4500,1.756600
5000,1.687500


# Inference

In [13]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [15]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model_path, sequence, max_length):
    
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    output = tokenizer.decode(final_outputs[0], skip_special_tokens=True)
    return output

In [16]:
model_path = "/content/drive/MyDrive/Prezent.ai/GPT-2/gpt-2-custom_q_and_a"
max_len = 50

def final_answer(model_path, sequence, max_len):
  result = generate_text(model_path, sequence, max_len)
  result = result.split('[Q]')[0].strip()
  try:
    start_index = result.index('[A]')
    end_index = result.index('.', start_index) + 1
    result = result[:end_index]
  except:
    pass
  return result

### Test examples

In [17]:
sequence1 = "what is an RNA-seq tool?"
answer = final_answer(model_path, sequence1, max_len)
print(answer)

what is an RNA-seq tool?
[A] RNA-seq is a highly complex, multidisciplinary technology, and its applications have evolved with the advancements in sequencing technologies.


In [18]:
sequence2 = "Which receptors are targeted by Tirzepatide?"
answer = final_answer(model_path, sequence2, max_len)
print(answer)

Which receptors are targeted by Tirzepatide?
[A] Tirzepatide is a dual-G protein coupled receptor tyrosine kinase inhibitor (DKK-I) developed to treat many types of asthma.


In [24]:
sequence3 = "What are TAMs in cancer therapy?"
answer = final_answer(model_path, sequence3, max_len)
print(answer)

What are TAMs in cancer therapy?
[A] TAMs are covalently binding to tumor suppressor genes.


In [25]:
sequence4 = "Is tofacitinib a JAK inhibitor?"
answer = final_answer(model_path, sequence4, max_len)
print(answer)

Is tofacitinib a JAK inhibitor?
[A] Tofacitinib (TKI) is a JAK inhibitor and has been shown to inhibit JAK transcription in vitro.


In [26]:
sequence5 = "What is the human proteoform project?"
answer = final_answer(model_path, sequence5, max_len)
print(answer)

What is the human proteoform project?
[A] To date, more than 3800 proteins have been analyzed in more than 55 large-scale proteomics projects worldwide, which range from basic catalytic pathway studies to large-scale synthetic biology
